## Setup / Install

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
import sys
sys.path.insert(1, '/usr/local/google/home/kmg/')
sys.path.insert(1, '/mnt/c/Users/Kevin Graney/SIG Groupings/')

from sig_groups.rider import Leader, Participant, RiderData, Match
from sig_groups.airtable import AirtableClient
from sig_groups.formatting import PrintAvailabilityTable, PrintRosters
from sig_groups.optimizer import AlgorithmTM, Params
from sig_groups.slack import PostRoster, PostRosterStatus
from sig_groups.ride import Ride, Roster, Rosters
from sig_groups.config import LoadConfigFile

## Parameters
These can be edited if needed.

In [ ]:
#@title Algorithm Parameters
params = Params()

params.start_ride = 5#@param {type:"integer"}
params.finalized_ride = 4#param {type:"integer"}

# Hard limits on group sizes.
params.min_group_size = 4#@param {type:"integer"}
params.max_group_size = 20#@param {type:"integer"}

# The maximum number of groups to assign each week.
params.max_groups = 8#@param {type:"integer"}

# Total number of rides in the program.
params.num_rides = 10#@param {type:"integer"}

# Maximum amount of time to run for, in seconds, per-pass.
params.time_limit = 10#@param {type:"integer"}

# Load Data

In [ ]:
config = LoadConfigFile("configs/2023.yaml")
rides = [Ride(x) for x in config.Rides()]
airtable_client = AirtableClient(config.Airtable(), config.Rides())
rider_data = RiderData(airtable_client.LoadLeaders(), airtable_client.LoadParticipants(), []) 
for m in config.Matches():
    rider_data.SetMatchScore(m['r1'], m['r2'], m['score'])     

In [ ]:
PrintAvailabilityTable(rider_data.AllRiders(), config.NumRides())

In [ ]:
prior_rosters = airtable_client.GetPriorRosters(rider_data)    

# Generate Rosters

In [ ]:
alg = AlgorithmTM(rider_data, prior_rosters, params)
rosters = alg.Solve()

In [ ]:
PrintRosters(rosters, rider_data)

In [ ]:
for r in rosters:
  if r.ride == 0:
    print(r )

In [ ]:
# map from ride -> pair
rides_together_through = defaultdict(lambda: defaultdict(lambda: 0))

rides_together = defaultdict(lambda: 0)
rides_together_so_far = defaultdict(lambda: 0)
match_hist = defaultdict(lambda: 0)
        
num_rides = config.NumRides()
for r in rosters:
  seen = set()
  for p1 in r.rider_ids:
    for p2 in r.rider_ids:
      canonical_pair = tuple(sorted((p1,p2)))
      if canonical_pair in seen:
        continue
      seen.add(canonical_pair)
      for i in range(r.ride, num_rides):
        rides_together_through[i][canonical_pair] += 1
      rides_together[(p1, p2)] += 1
      if r.ride <= params.start_ride:
        rides_together_so_far[(p1, p2)] += 1
      match_hist[rider_data.GetMatchScore(p1, p2)] += 1
    

#for k, v in rides_together_through.items():
#    print(k)
#    for k2, v2 in v.items():
#        print(k2, v2)
    
people = rider_data.AllRiders()
hist = defaultdict(lambda: 0)
for p1 in people:
  for p2 in people:
    canonical_pair = tuple(sorted((p1.id, p2.id)))
    if (p1 != p2):
      hist[rides_together_through[9][canonical_pair]] += 1

plt.bar(hist.keys(), hist.values())

In [ ]:
def PairFrequencyPlot(finalized, ride):
    final_pair_matrix = np.zeros(shape=(len(people), len(people)))
    modeled_pair_matrix = np.zeros(shape=(len(people), len(people)))
    for i, p1 in enumerate(people):
      for j, p2 in enumerate(people):
        canonical_pair = tuple(sorted((p1.id, p2.id)))
        if i < j:
            final_pair_matrix[j][i] = rides_together_through[finalized][canonical_pair]
        if i <= j:
            modeled_pair_matrix[i][j] = rides_together_through[ride][canonical_pair]
            modeled_pair_matrix[j][i] = rides_together_through[config.NumRides() - 1][canonical_pair]
            
           

    labels = [p.name for p in people]

    maxval = np.max([np.max(final_pair_matrix), np.max(modeled_pair_matrix)])        
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,20))
    ax1.set_title('Modeled to Finish / Through Ride %d' % (ride + 1))    
    ax1.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, rotation=90, fontsize='x-small')
    ax1.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, rotation=90, fontsize='x-small')
    ax1.set_yticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, fontsize='x-small')
    ax1.set_yticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, fontsize='x-small')  
    ax2.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=True, rotation=90, fontsize='x-small')
    ax2.set_xticks(np.arange(0, len(people), 1.0), labels=labels, minor=False, rotation=90, fontsize='x-small')

    #ax1.set_xticks(range(0, len(people)), labels=labels, minor=False, rotation=90, fontsize='xx-small')

    #ax1.set_xticklabels(labels, rotation=90, horizontalalignment='right')
    ax1.matshow(modeled_pair_matrix, vmin=0, vmax=maxval, cmap=plt.cm.viridis)
    ax2.set_title('Through Ride %d' % (finalized + 1))
    ax2.matshow(final_pair_matrix, vmin=0, vmax=maxval, cmap=plt.cm.viridis)
    #fig.colorbar(plot2)
    if ride <= finalized:
        fig.savefig('/tmp/pairings-%d.png' % ride, facecolor='green', transparent=False, bbox_inches='tight')
    else:
        fig.savefig('/tmp/pairings-%d.png' % ride, facecolor='yellow', transparent=False, bbox_inches='tight')
for i in range(0, config.NumRides()):
    PairFrequencyPlot(5, i)

In [ ]:
import imageio

with imageio.get_writer('mygif.gif', mode='I', fps=2) as writer:
    files = ['/tmp/pairings-%d.png' % i for i in range(0, config.NumRides())]
    for filename in files:
        image = imageio.imread(filename)
        writer.append_data(image)